## 用训练集训练模型，然后在测试集上面测试模型效果

In [1]:
import copy
import pandas as pd
import torch
import torch.nn as nn
import logging as log
from torch.optim import AdamW
from back_tool import seed_everything,train_model,test_model,processing_data,MyDataset
from models import MlpNeuralNet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tensor([[51.6201, 59.3296, 45.8101,  ..., 49.2737, 58.8827, 65.9218],
        [51.1547, 40.0693, 66.6282,  ..., 76.7898, 61.5473, 50.6928],
        [87.1711, 90.1275, 86.1134,  ..., 92.1345, 96.0673, 88.4188],
        ...,
        [48.7455, 53.0466, 86.0215,  ..., 58.4229, 45.4002, 55.5556],
        [86.3341, 87.9067, 85.9002,  ..., 87.6627, 88.5304, 90.5911],
        [ 3.1894,  7.4837,  8.2622,  ..., 54.5455, 58.6389, 57.5339]],
       dtype=torch.float64) tensor([1, 1, 0, 0, 3, 3, 3, 3, 2, 2, 1, 2, 0, 1, 0, 3, 3, 2, 3, 3, 2, 2, 0, 3,
        3, 2, 2, 0, 3, 2, 1, 2, 3, 3, 3, 2, 2, 3, 0, 3, 2, 3, 2, 2, 3, 2, 2, 0,
        3, 0, 3, 0, 2, 2, 3, 2, 2, 2, 2, 0, 3, 1, 0, 2, 3, 2, 3, 1, 3, 1, 2, 3,
        0, 2, 2, 3, 1, 0, 3, 1, 3, 1, 3, 2, 3, 1, 1, 3, 1, 3, 1, 1, 2, 2, 2, 2,
        3, 2, 1, 0, 3])


### 1.实验前准备

#### 1.1全局配置

In [2]:
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"
log.basicConfig(filename='log/log.txt', filemode='w', level=log.DEBUG, format=LOG_FORMAT, datefmt=DATE_FORMAT)

GLOB_CFG = { #训练的参数配置
    'seed': 2,
    'max_len': 100, #文本截断的最大长度
    'epochs': 150,
    'train_bs': 128, #batch_size，可根据自己的显存调整
    'valid_bs': 256,
    'test_bs': 256,
    'lr': 8e-6, #学习率
    'num_workers': 0,
    'weight_decay': 2e-4, #权重衰减，防止过拟合
    'device': 0,
    'train_data': '模式识别数据集/train_data.pkl',
    'test_data': None,
    'val_data': '模式识别数据集/validation_data.pkl',
    'log': log,
    'train': True, #训练阶段为true 提交测试阶段为false
    'processing_data': False,
    'seq_length':300,
    'sample_n': 200 # 抽取的样本条数，None表示抽取全部样本
}

seed_everything(GLOB_CFG['seed'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### 1.2 导入数据

In [3]:
if GLOB_CFG['processing_data']:
    processing_data('模式识别数据集', 'train', n=GLOB_CFG['sample_n'])
    processing_data('模式识别数据集', 'validation', n=GLOB_CFG['sample_n'])
train_df =  pd.read_pickle(GLOB_CFG['train_data'])
test_df = None
val_df = pd.read_pickle(GLOB_CFG['val_data'])
data_set = MyDataset(train_df, test_df, val_df, GLOB_CFG)

#### 实验一：多层感知机分类

#### 1.1 定义超参数

In [6]:
MLP_CFG = GLOB_CFG.copy()
log.basicConfig(filename='log/mlp_log.txt', filemode='w', level=log.DEBUG, format=LOG_FORMAT, datefmt=DATE_FORMAT)
MLP_CFG['log'] = log
MLP_CFG['hidden_dim'] = 128
MLP_CFG['feature_count'] = 1

#### 1.2 定义模型并训练

In [ ]:

model = MlpNeuralNet(MLP_CFG['seq_length']*MLP_CFG['feature_count'],\
    MLP_CFG['hidden_dim'],\
    data_set.get_label_num()).to(device)
optimizer = AdamW(model.parameters(), lr=MLP_CFG['lr'], weight_decay=MLP_CFG['weight_decay'])
criterion = nn.CrossEntropyLoss()

for epoch in range(MLP_CFG['epochs']):
        train_loss, train_acc = train_model(model,optimizer,  data_set,device,criterion, MLP_CFG)
        val_loss, val_acc = test_model(model, data_set,criterion, device,MLP_CFG)
        best_acc = 0
        if val_acc > best_acc:
            torch.save(model.state_dict(), './model_saved/{}_mlp_{}.pt'.format(epoch, round(val_acc, 4)))
            best_acc = val_acc

  0%|          | 0/1 [00:00<?, ?it/s]